In [1]:
import os
import sys
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras import utils
from keras.layers import Dense, Input, Flatten, LSTM, GRU
from keras.layers import Conv1D, MaxPooling1D, Embedding, GlobalAveragePooling1D
from keras.models import Model

Using TensorFlow backend.


In [2]:
BASE_DIR = '/home/ubuntu/'
GLOVE_DIR = BASE_DIR + '/glove.6B/'
TEXT_DATA_DIR = BASE_DIR + '/20_newsgroup/'
MAX_SEQUENCE_LENGTH = 25000
MAX_NB_WORDS = 50000
EMBEDDING_DIM = 50
VALIDATION_SPLIT = 0.1
GLOVE = True

In [3]:
if GLOVE:
    print('Indexing word vectors...')

    embeddings_index = {}
    f = open(os.path.join(GLOVE_DIR, 'glove.6B.50d.txt'))
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()

    print('Found %s word vectors.' % len(embeddings_index))

Indexing word vectors...
Found 400000 word vectors.


In [4]:
print('Processing text dataset')

authorlist = []
def load_data(data_type, full_doc=False):
    x = []
    y = []
    with open('../NLP/Gutenberg/Gutenberg/filenames_' +data_type+'.txt', 'r') as filenames:
        for filename in filenames:
            author = filename[:filename.index("_")]
            if author not in authorlist and len(authorlist) <10:
                authorlist.append(author)
            if author not in authorlist:
                continue
            with open('../NLP/Gutenberg/Gutenberg/'+data_type+'/'+ filename.strip(), 'r') as f:
                if full_doc:
                    data = f.read()
                    x.append(data)
                    y.append(authorlist.index(author))
                else:
                    data = sent_tokenize(f.read())
                    x += data
                    y += [authorlist.index(author)] * len(data)
        return [x, y]

texts, labels = load_data('train', full_doc=True)
test_texts, test_labels = load_data('test', full_doc=True)

print('Found %s texts.' % len(texts))
print('Found %s test texts.' % len(test_texts))

Processing text dataset
Found 90 texts.
Found 40 test texts.


In [5]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
test_sequences = tokenizer.texts_to_sequences(test_texts)

In [6]:
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 70545 unique tokens.


In [7]:
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
test = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)

labels = utils.to_categorical(np.asarray(labels))
test_labels = utils.to_categorical(np.asarray(test_labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Shape of data tensor: (90, 25000)
Shape of label tensor: (90, 10)


In [8]:
# split the data into a training set and a validation set
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
num_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = data[:-num_validation_samples]
y_train = labels[:-num_validation_samples]
x_val = data[-num_validation_samples:]
y_val = labels[-num_validation_samples:]

In [9]:
if GLOVE:
    print('Preparing embedding matrix.')

    num_words = min(MAX_NB_WORDS, len(word_index))
    embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
    for word, i in word_index.items():
        if i >= MAX_NB_WORDS:
            continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector

Preparing embedding matrix.


In [10]:
if GLOVE:
# load pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed
    embedding_layer = Embedding(num_words,
                                EMBEDDING_DIM,
                                weights=[embedding_matrix],
                                input_length=MAX_SEQUENCE_LENGTH,
                                trainable=False)
else:
    embedding_layer = Embedding(len(word_index) + 1,
                                EMBEDDING_DIM,
                                input_length=MAX_SEQUENCE_LENGTH)

In [11]:
print('Training model.')

# train a 1D convnet with global maxpooling
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = GRU(256, dropout=0.5, recurrent_dropout=0.5, return_sequences=True)(embedded_sequences)
x = GlobalAveragePooling1D()(x)
# x = LSTM(128, dropout=0.2, recurrent_dropout=0.2, return_sequences=True)(embedded_sequences)
# x = LSTM(32, dropout=0.2, recurrent_dropout=0.2)(x)
# x = Conv1D(128, 5, activation='relu')(embedded_sequences)
# x = MaxPooling1D(5)(x)
# x = Conv1D(128, 5, activation='relu')(x)
# x = MaxPooling1D(5)(x)
# x = Conv1D(128, 5, activation='relu')(x)
# x = MaxPooling1D(35)(x)
# x = Flatten()(x)
# x = Dense(128, activation='relu')(x)
preds = Dense(len(authorlist), activation='softmax')(x)

model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])

Training model.


In [ ]:
from keras.callbacks import ModelCheckpoint
checkpointer = ModelCheckpoint(filepath='/tmp/weights.hdf5', verbose=1, save_best_only=True)

model.fit(x_train, y_train,
          batch_size=10,
          epochs=64,
#           validation_split=0.1,
          validation_data=(x_val, y_val),
          callbacks=[checkpointer])

Train on 81 samples, validate on 9 samples
Epoch 1/64
81/81 [==============================] - 269s - loss: 2.3320 - acc: 0.0864 - val_loss: 2.3000 - val_acc: 0.0000e+00
Epoch 2/64
81/81 [==============================] - 256s - loss: 2.3302 - acc: 0.1358 - val_loss: 2.3067 - val_acc: 0.1111
Epoch 4/64
81/81 [==============================] - 257s - loss: 2.2087 - acc: 0.3086 - val_loss: 2.3082 - val_acc: 0.2222
Epoch 5/64
81/81 [==============================] - 257s - loss: 2.2317 - acc: 0.1728 - val_loss: 2.1945 - val_acc: 0.2222
Epoch 6/64
81/81 [==============================] - 254s - loss: 2.1803 - acc: 0.2222 - val_loss: 1.9810 - val_acc: 0.7778
Epoch 7/64
81/81 [==============================] - 256s - loss: 2.2029 - acc: 0.1728 - val_loss: 2.0253 - val_acc: 0.3333
Epoch 8/64
81/81 [==============================] - 256s - loss: 2.1369 - acc: 0.2222 - val_loss: 1.9780 - val_acc: 0.6667
Epoch 9/64
81/81 [==============================] - 265s - loss: 2.0968 - acc: 0.2716 - val_

In [13]:
model.summary()

score, acc = model.evaluate(pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH), test_labels, batch_size=64)
print('Test score:\n', score)
print('Test accuracy:\n', acc)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 25000)             0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 25000, 50)         2500000   
_________________________________________________________________
gru_1 (GRU)                  (None, 25000, 256)        235776    
_________________________________________________________________
global_average_pooling1d_1 ( (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                2570      
Total params: 2,738,346
Trainable params: 238,346
Non-trainable params: 2,500,000
_________________________________________________________________
40/40 [==============================] - 7s
Test score:
 1.66639518738
Test accuracy:
 0.425000011921
